# Langchain、Qdrant、OpenAIを使った質問応答システム

このノートブックでは、Langchain、ナレッジベースとしてのQdrant、OpenAI埋め込みを使用して質問応答システムを実装する方法を紹介します。Qdrantに馴染みがない場合は、[Getting_started_with_Qdrant_and_OpenAI.ipynb](Getting_started_with_Qdrant_and_OpenAI.ipynb) ノートブックを先に確認することをお勧めします。

このノートブックでは、以下のエンドツーエンドのプロセスを紹介します：
1. OpenAI APIを使用した埋め込みの計算
2. ナレッジベースを構築するためのQdrantローカルインスタンスへの埋め込みの保存
3. OpenAI APIを使用した生テキストクエリの埋め込みへの変換
4. 作成されたコレクション内で最近傍検索を実行してコンテキストを見つけるためのQdrantの使用
5. 与えられたコンテキスト内で答えを見つけるためのLLMへの問い合わせ

すべてのステップは、対応するLangchainメソッドの呼び出しに簡略化されます。

## 前提条件

この演習を行うために、いくつかの準備が必要です：

1. Qdrantサーバーインスタンス。今回はローカルのDockerコンテナを使用します。
2. ベクトルデータベースとやり取りするための[qdrant-client](https://github.com/qdrant/qdrant_client)ライブラリ。
3. フレームワークとしての[Langchain](https://github.com/hwchase17/langchain)。
3. [OpenAI API key](https://beta.openai.com/account/api-keys)。

### Qdrantサーバーの起動

Dockerコンテナで動作するローカルのQdrantインスタンスを使用します。起動する最も簡単な方法は、添付の[docker-compose.yaml]ファイルを使用して以下のコマンドを実行することです：

In [1]:
! docker-compose up -d

Starting qdrant_qdrant_1 ... 
ting qdrant_qdrant_1 ... done

サーバーが正常に起動したかどうかを確認するために、簡単な curl コマンドを実行して検証することができます：

In [2]:
! curl http://localhost:6333

{"title":"qdrant - vector search engine","version":"1.0.1"}

### 要件のインストール

このノートブックには明らかに `openai`、`langchain`、`qdrant-client` パッケージが必要です。

In [ ]:
! pip install openai qdrant-client "langchain==0.0.100" wget

### OpenAI APIキーの準備

OpenAI APIキーは、ドキュメントとクエリのベクトル化に使用されます。

OpenAI APIキーをお持ちでない場合は、[https://beta.openai.com/account/api-keys](https://beta.openai.com/account/api-keys)から取得できます。

キーを取得したら、以下のコマンドを実行して環境変数として`OPENAI_API_KEY`に追加してください：

In [ ]:
! export OPENAI_API_KEY="your API key"

In [4]:
# Test that your OpenAI API key is correctly set as an environment variable
# Note. if you run this notebook locally, you will need to reload your terminal and the notebook for the env variables to be live.
import os

# Note. alternatively you can set a temporary env variable like this:
# os.environ["OPENAI_API_KEY"] = "sk-xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx"

if os.getenv("OPENAI_API_KEY") is not None:
    print("OPENAI_API_KEY is ready")
else:
    print("OPENAI_API_KEY environment variable not found")

OPENAI_API_KEY is ready


## データの読み込み

このセクションでは、自然な質問とその回答を含むデータを読み込みます。すべてのデータは、Qdrantをナレッジベースとして使用するLangchainアプリケーションを作成するために使用されます。

In [5]:
import wget

# All the examples come from https://ai.google.com/research/NaturalQuestions
# This is a sample of the training set that we download and extract for some
# further processing.
wget.download("https://storage.googleapis.com/dataset-natural-questions/questions.json")
wget.download("https://storage.googleapis.com/dataset-natural-questions/answers.json")

100% [..............................................................................] 95372 / 95372

'answers.json'

In [6]:
import json

with open("questions.json", "r") as fp:
    questions = json.load(fp)

with open("answers.json", "r") as fp:
    answers = json.load(fp)

In [7]:
print(questions[0])

when is the last episode of season 8 of the walking dead


In [8]:
print(answers[0])

No . overall No. in season Title Directed by Written by Original air date U.S. viewers ( millions ) 100 `` Mercy '' Greg Nicotero Scott M. Gimple October 22 , 2017 ( 2017 - 10 - 22 ) 11.44 Rick , Maggie , and Ezekiel rally their communities together to take down Negan . Gregory attempts to have the Hilltop residents side with Negan , but they all firmly stand behind Maggie . The group attacks the Sanctuary , taking down its fences and flooding the compound with walkers . With the Sanctuary defaced , everyone leaves except Gabriel , who reluctantly stays to save Gregory , but is left behind when Gregory abandons him . Surrounded by walkers , Gabriel hides in a trailer , where he is trapped inside with Negan . 101 `` The Damned '' Rosemary Rodriguez Matthew Negrete & Channing Powell October 29 , 2017 ( 2017 - 10 - 29 ) 8.92 Rick 's forces split into separate parties to attack several of the Saviors ' outposts , during which many members of the group are killed ; Eric is critically injure

## チェーンの定義

LangchainはすでにQdrantと統合されており、与えられたドキュメントのリストに対してすべてのインデックス作成を実行します。私たちの場合、持っている回答のセットを保存することになります。

In [9]:
from langchain.vectorstores import Qdrant
from langchain.embeddings import OpenAIEmbeddings
from langchain import VectorDBQA, OpenAI

embeddings = OpenAIEmbeddings()
doc_store = Qdrant.from_texts(
    answers, embeddings, host="localhost" 
)

この段階で、可能なすべての回答はすでにQdrantに保存されているため、QAチェーン全体を定義することができます。

In [10]:
llm = OpenAI()
qa = VectorDBQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    vectorstore=doc_store,
    return_source_documents=False,
)

## データの検索

データがQdrantに格納されると、質問を開始できます。質問は自動的にOpenAIモデルによってベクトル化され、作成されたベクトルはQdrant内で一致する可能性のある回答を見つけるために使用されます。取得されると、最も類似した回答がOpenAI Large Language Modelに送信されるプロンプトに組み込まれます。すべてのサービス間の通信は以下のグラフに示されています：

![](https://qdrant.tech/articles_data/langchain-integration/flow-diagram.png)

In [11]:
import random

random.seed(52)
selected_questions = random.choices(questions, k=5)

In [12]:
for question in selected_questions:
    print(">", question)
    print(qa.run(question), end="\n\n")

> where do frankenstein and the monster first meet
 Victor and the Creature first meet in the mountains.

> who are the actors in fast and furious
 The actors in the Fast and Furious films are Vin Diesel, Paul Walker, Michelle Rodriguez, Jordana Brewster, Tyrese Gibson, Ludacris, Lucas Black, Sung Kang, Gal Gadot, Dwayne Johnson, Matt Schulze, Chad Lindberg, Johnny Strong, Eva Mendes, Devon Aoki, Nathalie Kelley, Bow Wow, Tego Calderón, Don Omar, Elsa Pataky, Kurt Russell, Nathalie Emmanuel, Scott Eastwood, Noel Gugliemi, Ja Rule, Thom Barry, Ted Levine, Minka Kelly, James Remar, Amaury Nolasco, Michael Ealy, MC Jin, Brian Goodman, Lynda Boyd, Jason Tobin, Neela, Liza Lapira, Alimi Ballard, Yorgo Constantine, Geoff Meed, Jeimy Osorio, Max William Crane, Charlie & Miller Kimsey, Eden Estrella, Romeo Santos, John Brotherton, Helen Mirren, Celestino Cornielle, Janmarco Santiago, Carlos De La Hoz, James Ayoub, Rick Yune, Cole Hauser, Brian Tee, John Ortiz, Luke Evans, Jason Statham, Charli

### カスタムプロンプトテンプレート

LangchainのStuffチェーンタイプは、質問とコンテキストドキュメントが組み込まれた特定のプロンプトを使用します。デフォルトのプロンプトは以下のようになっています：

```text
Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.
{context}
Question: {question}
Helpful Answer:
```

しかし、独自のプロンプトテンプレートを提供し、`stuff`チェーンタイプを使用しながらOpenAI LLMの動作を変更することができます。`{context}`と`{question}`をプレースホルダーとして保持することが重要です。

#### カスタムプロンプトの実験

異なるプロンプトテンプレートを使用して、モデルが以下のように動作するようにできます：
1. 答えを知っている場合は、一文で回答する。
2. 質問の答えがわからない場合は、ランダムな楽曲タイトルを提案する。

In [13]:
from langchain.prompts import PromptTemplate

In [14]:
custom_prompt = """
Use the following pieces of context to answer the question at the end. Please provide
a short single-sentence summary answer only. If you don't know the answer or if it's 
not present in given context, don't try to make up an answer, but suggest me a random 
unrelated song title I could listen to. 
Context: {context}
Question: {question}
Helpful Answer:
"""

In [15]:
custom_prompt_template = PromptTemplate(
    template=custom_prompt, input_variables=["context", "question"]
)

In [16]:
custom_qa = VectorDBQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    vectorstore=doc_store,
    return_source_documents=False,
    chain_type_kwargs={"prompt": custom_prompt_template},
)

In [17]:
random.seed(41)
for question in random.choices(questions, k=5):
    print(">", question)
    print(custom_qa.run(question), end="\n\n")

> what was uncle jesse's original last name on full house
Uncle Jesse's original last name on Full House was Cochran.

> when did the volcano erupt in indonesia 2018
No volcanic eruption is mentioned in the given context. Suggested Song: "Ring of Fire" by Johnny Cash.

> what does a dualist way of thinking mean
Dualist way of thinking means that the mind and body are separate entities, with the mind being a non-physical substance.

> the first civil service commission in india was set up on the basis of recommendation of
The first Civil Service Commission in India was not set up on the basis of a recommendation.

> how old do you have to be to get a tattoo in utah
In Utah, you must be at least 18 years old to get a tattoo.

